## PCO114 - Limpando e Transformando Dados com Python

> Nessa aula iremos utilizar a biblioteca Pandas do Python para nos conectarmos a uma fonte de dado e, atráves das funcionalidades da linguagem, iremos aplicar operações de limpeza e transformação nos dados.

Primeiramente, faça upload do arquivo 'dados_credito.xls' no menu lateral 'Arquivos'. O colab não mantem os arquivos carregados aqui, por ser um ambiente virtual de alocação dinâmica de recursos. Sendo assim, toda vez que voltar à um projeto do colab, é necessário recarregar os arquivos que são utilizados no seu código.

Vamos importar a biblioteca Pandas.

In [ ]:
import pandas as pd

Vamos carregar os nossos dados em um DataFrame do Pandas através da leitura do arquivo xls que foi carregado:

In [ ]:
df = pd.read_excel('dados_credito.xls')

Vamos verificar qual o número de linhas e colunas desse arquivo.

In [ ]:
df.shape

(30000, 25)

Ou seja, temos um arquivo com **30000 linhas** e **25 colunas**.

Os métodos `.info()` e `.head()` são muito utilizados para que tenhamos uma visão dos dados.

`.info()` fornece informações sobre todas as colunas e `.head()` mostra as primeiras linhas do DataFrame.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          30000 non-null  object
 1   LIMIT_BAL                   30000 non-null  int64 
 2   SEX                         30000 non-null  int64 
 3   EDUCATION                   30000 non-null  int64 
 4   MARRIAGE                    30000 non-null  int64 
 5   AGE                         30000 non-null  int64 
 6   PAY_1                       30000 non-null  object
 7   PAY_2                       30000 non-null  int64 
 8   PAY_3                       30000 non-null  int64 
 9   PAY_4                       30000 non-null  int64 
 10  PAY_5                       30000 non-null  int64 
 11  PAY_6                       30000 non-null  int64 
 12  BILL_AMT1                   30000 non-null  int64 
 13  BILL_AMT2                   30000 non-null  in

In [ ]:
#Por padrão, head traz as 5 primeiras linhas do arquivo
#Esse valor pode ser especificado dentro dos parênteses
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


# Vamos agora verificar a integridade básica dos dados. 

Nos foi dito que a coluna 'ID' contém valores únicos que identificam os clientes e que cada linha desse conjunto de dados representa um cliente. Devemos verificar se isso é verdade, analisando se essa coluna contém valores que não se repetem.**negrito**

In [ ]:
df['ID'].nunique()

29687

O comando acima nos diz que existem **29687 linhas** com ID que possuem valores únicos. Porém, nosso arquivo contém 30000 linhas, o que indica que alguns registros contém valores repetidos para o 'ID' do cliente.

Sendo assim, precisamos identificar quais IDs estão duplicados e quantas vezes isso ocorre.

Usaremos o método `.value_counts` na coluna ID para começar a identificar esses problemas. Esse método é semelhante ao **group by/count** de um comando SQL. Ele irá exibir os IDs e a frequência com que ocorrem.

In [ ]:
id_cont = df['ID'].value_counts()
id_cont.head() #exibindo somente as 5 primeiras linhas

823267c1-561b    2
ffb9a143-c375    2
249b09c0-f4c7    2
0ad143f2-bd95    2
a7559852-f405    2
Name: ID, dtype: int64

Vamos aplicar o método `.value_counts` na variável id_cont para obter a quantidade de entradas que foram duplicadas.

In [ ]:
id_cont.value_counts()

1    29374
2      313
Name: ID, dtype: int64

Dessa forma podemos notar que a maioria dos IDs aparece uma única vez, mas 313 desses IDs aparecem 2 vezes. É possível perceber que nenhum ID aparece mais de 2 vezes.

Partiremos agora para a limpeza desses dados duplicados.

# Criando máscaras booleanas para limpar os dados

Também conhecida como máscara lógica, uma máscara booleana serve para filtrar um array ou série obedecendo alguma condição.

Para exemplificar, vamos utilizar um array genérico antes de aplicarmos as mudanças no nosso conjunto de dados, utilizando a biblioteca `numpy`.

In [ ]:
import numpy as np

Vamos criar um array de 100 números inteiros de forma aleatória. A função de aleatoriedade do numpy permite que uma semente (**seed**) seja especificada para que experimentos que utilizam números aleatórios possam ser reproduzidos.

In [ ]:
np.random.seed(seed=24)

inteiros_aleatorios = np.random.randint(low=1,high=5,size=100)

#vamos verificar os 5 primeiros itens
inteiros_aleatorios[:5]

array([3, 4, 1, 4, 2])

Suponha que queremos identificar todos os valores dentro desse array que sejam iguais a 3. Nossa máscara booleana seria algo como:

In [ ]:
igual_a_3 = inteiros_aleatorios == 3

#vamos verificar os 5 primeiros itens
igual_a_3[:5]

array([ True, False, False, False, False])

Podemos agora utilizar essa máscara para indexar o array original e obter todas as ocorrências do valor 3.

In [ ]:
inteiros_aleatorios[igual_a_3]

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

Dado o exemplo acima, vamos aplicar o conceito das máscaras booleanas no conjunto de dados. Para isso, vamos encontrar na variável `id_cont` valores de contagem que sejam iguais a 2. Atribuiremos os IDs duplicados a uma variável chamada `dup_mask` e exibiremos os cinco primeiros valores:

In [ ]:
dup_mask = id_cont == 2
dup_mask[0:5]

823267c1-561b    True
ffb9a143-c375    True
249b09c0-f4c7    True
0ad143f2-bd95    True
a7559852-f405    True
Name: ID, dtype: bool

Com a máscara criada, vamos selecionar os IDs duplicados. Os IDs aparecem como o índice da série `id_cont`, sendo possível então que possamos aplicar a máscara lógica para realizar a seleção dos IDs duplicados:

In [ ]:
dup_ids = id_cont.index[dup_mask]

Vamos converter `dup_ids` em uma lista e verificar o tamanho dela:

In [ ]:
dup_ids = list(dup_ids)
len(dup_ids)

313

Podemos observar que esse valor corresponde ao número de IDs identificdos como duplicatas. Sendo assim, vamos agora utilizar essa lista para analisar os dados presentes nesses IDs.

Primeiramente, vamos ver o conteúdo de `dup_ids` (5 primeiras linhas):

In [ ]:
dup_ids[:5]

['823267c1-561b',
 'ffb9a143-c375',
 '249b09c0-f4c7',
 '0ad143f2-bd95',
 'a7559852-f405']

São os IDs duplicados. Vamos examinar os valores das outras colunas que pertencem à esses IDs. Para isso, utilizaremos os métodos `.isin` e `.loc` do DataFrame.

Vamos utilizar os 3 primeiros valores da nossa lista. 

Primeiro, queremos encontrar as linhas que contenham esses IDs. Faremos isso com a função `.isin` na coluna ID do DataFrame. Ela irá criar uma outra máscara lógica para exibir as linhas que contém aqueles IDs da lista. Usaremos também a função `.loc` que irá selecionar as linhas que retornarem `True` na máscara lógica.

In [ ]:
df.loc[df['ID'].isin(dup_ids[0:3]), :]

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
5753,823267c1-561b,330000,2,2,2,30,-2,-2,-2,-2,-2,-2,0,0,0,0,0,0,0,0,0,0,0,0,1
5853,823267c1-561b,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6118,249b09c0-f4c7,360000,1,1,2,29,-1,-1,-1,-2,-1,0,2273,1170,0,0,7972,8275,1175,0,0,7972,435,2947,0
6218,249b09c0-f4c7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8746,ffb9a143-c375,80000,1,1,2,27,0,0,0,0,0,0,42653,46609,47306,51179,52073,52833,5000,1801,5000,2035,2100,6200,0
8846,ffb9a143-c375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Ao analisar os dados acima, vemos que para cada linha duplicada, existe uma linha que parece ter dados válidos e outra com valores 0. Vamos considerar que os valores 0 nesse conjunto de dados são um erro da extração. Sendo assim, o critério que vamos adotar para limpar os valores duplicados será o de linhas que possuem valores 0 em todas as suas colunas (exceto no ID).

Para isso, vamos criar uma outra máscara que será aplicada à todo o DataFrame, então ela precisa ser uma matriz do mesmo tamanho do DataFrame:

In [ ]:
df_zero = df == 0

Nas próximas etapas usaremos o `df_zero` para criar uma série booleana que identifique cada linha em que todos os elementos a partir da segunda coluna sejam 0. Teremos que indexar df_zero usano o método de indexação de inteiros `.iloc`.

In [ ]:
atributos_zero_mask = df_zero.iloc[:, 1:].all(axis=1)

Vamos calcular a soma dessa série:

In [ ]:
sum(atributos_zero_mask)

315

Essa saída nos diz que 315 linhas têm zeros para cada coluna exceto a primeira. Esse número é maior do que o número de IDs duplicados, logo, se excluirmos todas as "linhas de 0", podemos nos livrar das duplicatas.

Vamos limpar o DataFrame original com essa máscara:

In [ ]:
df_limpo = df.loc[~atributos_zero_mask, :].copy()

Nesse trecho o que fizemos foi aplicar a negação da máscara, ou seja, selecionamos os valores que são diferentes de 0. O operador `.copy()` faz uma cópia do DataFrame.

Analisando agora o DataFrame resultante através do operador `.shape`:

In [ ]:
df_limpo.shape

(29685, 25)

Vemos que eliminamos com sucesso as duplicatas.

In [ ]:
df_limpo['ID'].nunique()

29685